# Ordered Logit/Probit: Solutions

**Tutorial Series**: Discrete Choice Econometrics with PanelBox

**Notebook**: 07 - Ordered Models (Solutions)

**Author**: PanelBox Contributors

**Date**: 2026-02-17

---

This notebook contains complete solutions for the exercises in `07_ordered_models.ipynb`.

In [ ]:
# Setup (same as main notebook)
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, chi2
from scipy.special import expit
import statsmodels.api as sm

from panelbox.models.discrete.ordered import (
    OrderedLogit, OrderedProbit, RandomEffectsOrderedLogit
)
from panelbox.models.discrete.multinomial import MultinomialLogit

warnings.filterwarnings('ignore')
np.random.seed(42)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

DATA_DIR = Path("..") / "data"
OUTPUT_DIR = Path("..") / "outputs"
FIG_DIR = OUTPUT_DIR / "figures"
TABLE_DIR = OUTPUT_DIR / "tables"
FIG_DIR.mkdir(parents=True, exist_ok=True)
TABLE_DIR.mkdir(parents=True, exist_ok=True)

RATING_LABELS = {0: 'Poor', 1: 'Fair', 2: 'Good', 3: 'Excellent'}
RATING_COLORS = ['#e74c3c', '#f39c12', '#3498db', '#2ecc71']

# Load data
data = pd.read_csv(DATA_DIR / "credit_rating.csv")
exog_vars = ['income', 'debt_ratio', 'age', 'size', 'profitability']
y = data['rating'].values
X = data[exog_vars].values
groups = data['id'].values
time = data['year'].values
K = len(exog_vars)
n_obs = len(y)

# Estimate base ordered logit (needed for all exercises)
model_ologit = OrderedLogit(endog=y, exog=X, groups=groups, time=time, n_categories=4)
model_ologit.exog_names = exog_vars
results_ologit = model_ologit.fit()

# AIC/BIC for base model
n_params_ologit = K + 3
aic_logit = -2 * results_ologit.llf + 2 * n_params_ologit
bic_logit = -2 * results_ologit.llf + np.log(n_obs) * n_params_ologit

print("Setup complete. Ordered Logit estimated.")
print(f"Log-L: {results_ologit.llf:.2f}")
print(f"AIC: {aic_logit:.2f}, BIC: {bic_logit:.2f}")

---

## Exercise 1: Ordered Logit vs Multinomial Logit (Medium)

**Task**: Compare Ordered Logit with Multinomial Logit on ordinal data.

In [ ]:
# Exercise 1 Solution

# Step 1: Estimate Multinomial Logit
model_mnl = MultinomialLogit(
    endog=y,
    exog=X,
    n_alternatives=4,
    base_alternative=0
)
results_mnl = model_mnl.fit()

print("=== Multinomial Logit Results ===")
print(results_mnl.summary())

In [ ]:
# Step 2: Compare parameter counts and fit
n_params_mnl = (4 - 1) * K  # (J-1) x K = 3 x 5 = 15
n_params_ologit = K + 3     # K + (J-1) = 5 + 3 = 8

aic_mnl = -2 * results_mnl.llf + 2 * n_params_mnl
bic_mnl = -2 * results_mnl.llf + np.log(n_obs) * n_params_mnl

print("=== Model Comparison: Ordered Logit vs Multinomial Logit ===")
print(f"\n{'Metric':<25} {'Ordered Logit':>15} {'MNL':>15}")
print("-" * 60)
print(f"{'N parameters':<25} {n_params_ologit:>15d} {n_params_mnl:>15d}")
print(f"{'Log-likelihood':<25} {results_ologit.llf:>15.2f} {results_mnl.llf:>15.2f}")
print(f"{'AIC':<25} {aic_logit:>15.2f} {aic_mnl:>15.2f}")
print(f"{'BIC':<25} {bic_logit:>15.2f} {bic_mnl:>15.2f}")

print(f"\nParameter savings: {n_params_mnl - n_params_ologit} fewer parameters in Ordered Logit")
print(f"  Ordered Logit: {n_params_ologit} = {K} betas + {3} cutpoints")
print(f"  MNL:           {n_params_mnl} = {K} betas x {3} alternatives")

print(f"\n=== Answers ===")
print(f"1. Ordered Logit is more parsimonious ({n_params_ologit} vs {n_params_mnl} parameters).")
print(f"2. AIC: {'OLogit' if aic_logit < aic_mnl else 'MNL'} is preferred.")
print(f"   BIC: {'OLogit' if bic_logit < bic_mnl else 'MNL'} is preferred.")
print(f"3. Prefer MNL when ordering assumption is questionable or")
print(f"   when effects genuinely differ across category transitions.")

---

## Exercise 2: Cutpoints Exploration (Easy)

**Task**: Merge categories and compare cutpoints.

In [ ]:
# Exercise 2 Solution

# Step 1: Full model (already estimated)
print("=== 4-Category Model ===")
print(f"Cutpoints: {results_ologit.cutpoints}")
print(f"Betas: {results_ologit.beta}")
print(f"Log-L: {results_ologit.llf:.4f}")

# Step 2: Merge categories 0 and 1 (Poor + Fair -> "Low")
y_merged = y.copy()
y_merged[y_merged == 1] = 0  # Fair becomes Low (0)
y_merged[y_merged == 2] = 1  # Good becomes 1
y_merged[y_merged == 3] = 2  # Excellent becomes 2

print(f"\nMerged category distribution:")
for val, label in {0: 'Low (Poor+Fair)', 1: 'Good', 2: 'Excellent'}.items():
    count = (y_merged == val).sum()
    print(f"  {val} ({label}): {count} ({count/len(y_merged):.1%})")

# Step 3: Re-estimate with 3 categories
model_3cat = OrderedLogit(
    endog=y_merged, exog=X, groups=groups, time=time, n_categories=3
)
model_3cat.exog_names = exog_vars
results_3cat = model_3cat.fit()

print(f"\n=== 3-Category Model ===")
print(f"Cutpoints: {results_3cat.cutpoints}")
print(f"Betas: {results_3cat.beta}")
print(f"Log-L: {results_3cat.llf:.4f}")

In [ ]:
# Step 4: Compare
print("=== Comparison ===")
print(f"\n{'Variable':<15} {'4-cat':>10} {'3-cat':>10} {'Change%':>10}")
print("-" * 50)
for k, var in enumerate(exog_vars):
    b4 = results_ologit.beta[k]
    b3 = results_3cat.beta[k]
    change = ((b3 - b4) / abs(b4)) * 100 if abs(b4) > 1e-6 else np.nan
    print(f"{var:<15} {b4:>10.4f} {b3:>10.4f} {change:>+9.1f}%")

print(f"\nCutpoints:")
print(f"  4-cat: {results_ologit.cutpoints}")
print(f"  3-cat: {results_3cat.cutpoints}")
print(f"\nLog-likelihood:")
print(f"  4-cat: {results_ologit.llf:.4f}")
print(f"  3-cat: {results_3cat.llf:.4f}")

print(f"\n=== Answers ===")
print(f"1. The 3-cat model has only 2 cutpoints (vs 3).")
print(f"   The remaining cutpoints roughly correspond to the 4-cat upper cutpoints.")
print(f"2. Beta coefficients change somewhat because the latent variable scale shifts.")
print(f"3. The 3-cat log-likelihood differs because information is lost by merging.")

---

## Exercise 3: Brant Test Interpretation (Medium)

**Task**: Interpret the Brant test results.

In [ ]:
# Exercise 3 Solution

# Step 1: Run Brant test (same as Section 5 of main notebook)
X_with_const = sm.add_constant(X)
n_categories = 4
n_thresholds = 3

binary_betas = []
binary_vcovs = []

for j in range(n_thresholds):
    y_binary = (y > j).astype(int)
    logit_model = sm.Logit(y_binary, X_with_const)
    res_binary = logit_model.fit(disp=False)
    binary_betas.append(res_binary.params[1:])
    binary_vcovs.append(res_binary.cov_params()[1:, 1:])

beta_matrix = np.array(binary_betas)

# Per-variable Brant test
print("=== Brant Test Per Variable ===")
print(f"\n{'Variable':<15} {'chi2':>10} {'df':>5} {'p-value':>10} {'Conclusion':>15}")
print("-" * 60)

rejected_vars = []

for k, var in enumerate(exog_vars):
    betas_k = beta_matrix[:, k]
    diffs = betas_k[1:] - betas_k[0]
    var_diffs = np.array([
        binary_vcovs[j+1][k, k] + binary_vcovs[0][k, k]
        for j in range(len(diffs))
    ])
    chi2_k = np.sum(diffs**2 / var_diffs)
    df_k = len(diffs)
    p_k = 1 - chi2.cdf(chi2_k, df_k)
    conclusion = 'Reject' if p_k < 0.05 else 'Fail to reject'
    print(f"{var:<15} {chi2_k:>10.3f} {df_k:>5d} {p_k:>10.4f} {conclusion:>15}")
    if p_k < 0.05:
        rejected_vars.append(var)

print(f"\nVariables violating proportional odds: {rejected_vars if rejected_vars else 'None'}")

In [ ]:
# Step 3: Inspect coefficients across thresholds
print("=== Coefficient Comparison Across Thresholds ===")
print(f"\n{'Variable':<15} {'P(y>0)':>10} {'P(y>1)':>10} {'P(y>2)':>10} {'Max diff':>10}")
print("-" * 60)
for k, var in enumerate(exog_vars):
    betas_k = beta_matrix[:, k]
    max_diff = np.max(betas_k) - np.min(betas_k)
    print(f"{var:<15} {betas_k[0]:>10.4f} {betas_k[1]:>10.4f} {betas_k[2]:>10.4f} {max_diff:>10.4f}")

print(f"\n=== Answers ===")
print(f"1. If debt_ratio violates proportional odds, it means the effect")
print(f"   of debt on the log-odds of better ratings varies by threshold.")
print(f"   For example, debt may matter more for the poor-to-fair transition")
print(f"   than for the good-to-excellent transition.")
print(f"")
print(f"2. If proportional odds is rejected:")
print(f"   - Generalized Ordered Logit (variable-specific cutpoints)")
print(f"   - Partial proportional odds (relax only for violating variables)")
print(f"   - Multinomial Logit (if ordering assumption itself is questionable)")
print(f"")
print(f"3. Yes, it's common for some variables to satisfy proportional odds")
print(f"   while others violate it. The per-variable Brant test identifies which.")

---

## Exercise 4: Marginal Effects Ambiguity (Medium)

**Task**: Demonstrate that intermediate category marginal effects can change sign.

In [ ]:
# Exercise 4 Solution

beta = results_ologit.beta
cutpoints = results_ologit.cutpoints
cutpoints_ext = np.concatenate([[-np.inf], cutpoints, [np.inf]])

def logistic_pdf(z):
    F = expit(z)
    return F * (1 - F)

# Step 1: Compute ME of income on each category at different income levels
base = data[exog_vars].mean().values
income_range = np.linspace(8, 13, 200)

me_by_income = np.zeros((200, 4))
for i, inc in enumerate(income_range):
    profile = base.copy()
    profile[0] = inc
    lp = profile @ beta
    for j in range(4):
        z_lo = cutpoints_ext[j] - lp
        z_hi = cutpoints_ext[j+1] - lp
        pdf_lo = logistic_pdf(z_lo) if np.isfinite(z_lo) else 0
        pdf_hi = logistic_pdf(z_hi) if np.isfinite(z_hi) else 0
        me_by_income[i, j] = beta[0] * (pdf_lo - pdf_hi)

print("=== Marginal Effect of Income on Each Category ===")
print("\nAt different income levels (other variables at means):")
for inc_check in [8.5, 9.5, 10.5, 11.5, 12.5]:
    idx = np.argmin(np.abs(income_range - inc_check))
    me_str = ', '.join([f"{RATING_LABELS[j]}={me_by_income[idx, j]:+.4f}" for j in range(4)])
    print(f"  income={inc_check:.1f}: {me_str}")

In [ ]:
# Step 2: Plot ME across income levels
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Panel A: ME of income on each category
ax = axes[0]
for j in range(4):
    ax.plot(income_range, me_by_income[:, j], linewidth=2,
            color=RATING_COLORS[j], label=RATING_LABELS[j])
ax.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax.axvline(x=base[0], color='gray', linestyle=':', alpha=0.7, label='Mean income')
ax.set_xlabel('Log Firm Income')
ax.set_ylabel('Marginal Effect dP/d(income)')
ax.set_title('ME of Income on P(rating=j)\nVarying the Evaluation Point', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel B: Verify sum-to-zero at each income level
ax = axes[1]
me_sum = me_by_income.sum(axis=1)
ax.plot(income_range, me_sum, 'k-', linewidth=2)
ax.axhline(y=0, color='red', linestyle='--', linewidth=1)
ax.set_xlabel('Log Firm Income')
ax.set_ylabel('Sum of ME across categories')
ax.set_title('Sum-to-Zero Verification\n(should be zero everywhere)', fontweight='bold')
ax.set_ylim(-0.001, 0.001)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n=== Key Observation ===")
# Check if intermediate categories change sign
for j in [1, 2]:  # Fair and Good (intermediate)
    signs = np.sign(me_by_income[:, j])
    if len(np.unique(signs[signs != 0])) > 1:
        print(f"  {RATING_LABELS[j]}: ME CHANGES SIGN across income range!")
    else:
        sign_label = 'positive' if signs[signs != 0][0] > 0 else 'negative'
        print(f"  {RATING_LABELS[j]}: ME is consistently {sign_label}")

print("\nThe intermediate categories (Fair, Good) can exhibit sign changes.")
print("This means the same variable can increase P(Good) at low income")
print("but decrease it at high income. This is a unique feature of ordered models.")

---

## Exercise 5: RE vs Pooled Ordered Logit (Hard)

**Task**: Evaluate unobserved heterogeneity.

In [ ]:
# Exercise 5 Solution

# Step 1: Estimate RE model
print("Fitting Random Effects Ordered Logit...")
model_re = RandomEffectsOrderedLogit(
    endog=y, exog=X, groups=groups, time=time,
    n_categories=4, quadrature_points=12
)
model_re.exog_names = exog_vars
results_re = model_re.fit(maxiter=500)

# Compare coefficients
print("\n=== Pooled vs RE Ordered Logit ===")
print(f"\n{'Variable':<15} {'Pooled':>10} {'RE':>10} {'Change%':>10}")
print("-" * 50)
for k, var in enumerate(exog_vars):
    b_pooled = results_ologit.beta[k]
    b_re = results_re.beta[k]
    change = ((b_re - b_pooled) / abs(b_pooled)) * 100 if abs(b_pooled) > 1e-6 else np.nan
    print(f"{var:<15} {b_pooled:>10.4f} {b_re:>10.4f} {change:>+9.1f}%")

In [ ]:
# Step 2: LR test for sigma_alpha
lr_stat = -2 * (results_ologit.llf - results_re.llf)

# Boundary problem: sigma_alpha >= 0, so we use mixture chi2
# P-value = 0.5 * P(chi2_0 > LR) + 0.5 * P(chi2_1 > LR)
# = 0.5 * I(LR > 0) + 0.5 * (1 - chi2.cdf(LR, 1))
# Simplified: p = 0.5 * (1 - chi2.cdf(LR, 1)) when LR > 0
p_lr = 0.5 * (1 - chi2.cdf(lr_stat, 1)) if lr_stat > 0 else 1.0

print(f"=== Likelihood Ratio Test for sigma_alpha ===")
print(f"\nH0: sigma_alpha = 0 (no unobserved heterogeneity)")
print(f"H1: sigma_alpha > 0")
print(f"\nLog-L (pooled):  {results_ologit.llf:.4f}")
print(f"Log-L (RE):      {results_re.llf:.4f}")
print(f"LR statistic:    {lr_stat:.4f}")
print(f"p-value (mixture chi2): {p_lr:.4f}")
print(f"Conclusion: {'Reject H0 -> RE model preferred' if p_lr < 0.05 else 'Fail to reject -> pooled is adequate'}")

# Step 3: Intraclass correlation
sigma = results_re.sigma_alpha
rho = sigma**2 / (sigma**2 + np.pi**2 / 3)

print(f"\n=== Intraclass Correlation ===")
print(f"sigma_alpha = {sigma:.4f}")
print(f"sigma_alpha^2 = {sigma**2:.4f}")
print(f"Logistic variance (pi^2/3) = {np.pi**2/3:.4f}")
print(f"rho = sigma^2 / (sigma^2 + pi^2/3) = {rho:.4f}")
print(f"\nInterpretation:")
print(f"  {rho:.1%} of the total latent variable variance is due to")
print(f"  firm-specific unobserved heterogeneity (e.g., management quality,")
print(f"  brand reputation, governance structure).")
if rho > 0.1:
    print(f"  This is substantial — ignoring it (pooled model) may bias estimates.")
else:
    print(f"  This is modest — the pooled model may be adequate.")

In [ ]:
# Visualize: coefficient comparison and predicted probability differences
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Panel A: Coefficient comparison
x = np.arange(len(exog_vars))
width = 0.35
axes[0].bar(x - width/2, results_ologit.beta, width, label='Pooled',
            color='#3498db', alpha=0.8, edgecolor='black')
axes[0].bar(x + width/2, results_re.beta, width, label='RE',
            color='#2ecc71', alpha=0.8, edgecolor='black')
axes[0].axhline(y=0, color='black', linewidth=0.8, alpha=0.5)
axes[0].set_xlabel('Variable')
axes[0].set_ylabel('Coefficient')
axes[0].set_title('Pooled vs RE Coefficients', fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(exog_vars, rotation=30)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Panel B: AIC/BIC comparison
n_params_re = K + 3 + 1
aic_re = -2 * results_re.llf + 2 * n_params_re
bic_re = -2 * results_re.llf + np.log(n_obs) * n_params_re

metrics = ['Log-L', 'AIC', 'BIC']
pooled_vals = [results_ologit.llf, aic_logit, bic_logit]
re_vals = [results_re.llf, aic_re, bic_re]
x2 = np.arange(len(metrics))

axes[1].bar(x2 - width/2, pooled_vals, width, label='Pooled',
            color='#3498db', alpha=0.8, edgecolor='black')
axes[1].bar(x2 + width/2, re_vals, width, label='RE',
            color='#2ecc71', alpha=0.8, edgecolor='black')
axes[1].set_xlabel('Metric')
axes[1].set_ylabel('Value')
axes[1].set_title('Model Fit Comparison', fontweight='bold')
axes[1].set_xticks(x2)
axes[1].set_xticklabels(metrics)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.suptitle('Pooled vs Random Effects Ordered Logit',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"\nAIC: Pooled = {aic_logit:.2f}, RE = {aic_re:.2f} -> {'RE' if aic_re < aic_logit else 'Pooled'} preferred")
print(f"BIC: Pooled = {bic_logit:.2f}, RE = {bic_re:.2f} -> {'RE' if bic_re < bic_logit else 'Pooled'} preferred")

---

**End of Solutions**